<a href="https://colab.research.google.com/github/testuayun/NWU-nCov-auto-report/blob/master/Fine_tune_PaliGemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## PaliGemma Fine-tuning

In this notebook, we will fine-tune [pretrained PaliGemma](https://huggingface.co/google/paligemma2-3b-pt-448) on a small split of [VQAv2](https://huggingface.co/datasets/HuggingFaceM4/VQAv2) dataset. Let's get started by installing necessary libraries.

In [1]:
!pip install -q -U datasets bitsandbytes peft git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


We will authenticate to access the model using `notebook_login()`.

In [2]:
from huggingface_hub import notebook_login
notebook_login()

Let's load the dataset.

In [3]:
from datasets import load_dataset
ds = load_dataset('merve/vqav2-small', split="validation")
ds = ds.train_test_split(test_size=0.1)
train_ds = ds["train"]
val_ds = ds["test"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/403 [00:00<?, ?B/s]

validation-00000-of-00007.parquet:   0%|          | 0.00/484M [00:00<?, ?B/s]

validation-00001-of-00007.parquet:   0%|          | 0.00/486M [00:00<?, ?B/s]

validation-00002-of-00007.parquet:   0%|          | 0.00/483M [00:00<?, ?B/s]

validation-00003-of-00007.parquet:   0%|          | 0.00/486M [00:00<?, ?B/s]

validation-00004-of-00007.parquet:   0%|          | 0.00/475M [00:00<?, ?B/s]

validation-00005-of-00007.parquet:   0%|          | 0.00/484M [00:00<?, ?B/s]

validation-00006-of-00007.parquet:   0%|          | 0.00/479M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/21435 [00:00<?, ? examples/s]

In [4]:
split_ds = ds.train_test_split(test_size=0.9) # we'll use a very small split for demo
train_ds = split_ds["test"]

AttributeError: 'DatasetDict' object has no attribute 'train_test_split'

In [5]:
train_ds

Dataset({
    features: ['multiple_choice_answer', 'question', 'image'],
    num_rows: 19291
})

Our dataset is a very general one and similar to many datasets that PaliGemma was trained with. In this case, we do not need to fine-tune the image encoder, the multimodal projector but we will only fine-tune the text decoder.

In [6]:
from transformers import PaliGemmaProcessor
model_id ="google/paligemma-3b-ft-vqav2-448" # or your favorite PaliGemma

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [7]:
from transformers import PaliGemmaForConditionalGeneration
import torch
device = "cuda"
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, torch_dtype=torch.bfloat16).to(device)

for param in model.vision_tower.parameters():
    param.requires_grad = False

for param in model.multi_modal_projector.parameters():
    param.requires_grad = False


config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [8]:
from transformers import BitsAndBytesConfig, PaliGemmaForConditionalGeneration
from peft import get_peft_model, LoraConfig

bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16)

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, device_map="auto")#, quantization_config=bnb_config)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 11,298,816 || all params: 2,935,650,032 || trainable%: 0.3849


Alternatively, if you want to do LoRA and QLoRA fine-tuning, you can run below cells to load the adapter either in full precision or quantized.

We need to take tokens to same dtype as model so need to store it as a variable.

In [9]:
DTYPE = model.dtype

Load the processor to preprocess the dataset.

In [10]:
processor = PaliGemmaProcessor.from_pretrained(model_id)

preprocessor_config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

We will preprocess our examples. We need to prepare a prompt template and pass the text input inside, pass it with batches of images to processor. Then we will set the pad tokens and image tokens to -100 to let the model ignore them. We will pass our preprocessed input as labels to make the model learn how to generate responses.

In [11]:
import torch

image_token = processor.tokenizer.convert_tokens_to_ids("<image>")
def collate_fn(examples):
  texts = ["<image>answer en " + example["question"] for example in examples]
  labels= [example['multiple_choice_answer'] for example in examples]
  images = [example["image"].convert("RGB") for example in examples]
  tokens = processor(text=texts, images=images, suffix=labels,
                    return_tensors="pt", padding="longest")

  tokens = tokens.to(DTYPE).to(device)
  return tokens


We will now initialize the `TrainingArguments`.

In [12]:
from transformers import TrainingArguments
args=TrainingArguments(
            num_train_epochs=1,
            remove_unused_columns=False,
            per_device_train_batch_size=1,
            gradient_accumulation_steps=4,
            warmup_steps=2,
            learning_rate=2e-5,
            weight_decay=1e-6,
            adam_beta2=0.999,
            logging_steps=100,
            optim="paged_adamw_8bit", # you can use paged optimizers like paged_adamw_8bit for QLoRA
            save_strategy="steps",
            save_steps=1000,
            save_total_limit=1,
            output_dir="paligemma_vqav2",
            bf16=True,
            report_to=["tensorboard"],
            dataloader_pin_memory=False
        )


We can now start training.

In [13]:
from transformers import Trainer

trainer = Trainer(
        model=model,
        train_dataset=train_ds ,
        eval_dataset=val_ds,
        data_collator=collate_fn,
        args=args
        )


LoRA with bsz of 2 works on A100 Colab. You can apply gradient accumulation (which is enabled in this notebook) to simulate larger batch sizes.
Currently there's an issue with QLoRA, we are investigating and will solve soon.

In [14]:
trainer.train()

Step,Training Loss
100,0.277300
200,0.262000
300,0.200000
400,0.251600
500,0.238900
600,0.260900
700,0.234600
800,0.245100
900,0.178000
1000,0.188300


Step,Training Loss
100,0.277300
200,0.262000
300,0.200000
400,0.251600
500,0.238900
600,0.260900
700,0.234600
800,0.245100
900,0.178000
1000,0.188300


TrainOutput(global_step=4822, training_loss=0.21312973272844946, metrics={'train_runtime': 6560.2069, 'train_samples_per_second': 2.941, 'train_steps_per_second': 0.735, 'total_flos': 2.891077359745473e+17, 'train_loss': 0.21312973272844946, 'epoch': 0.9998444870665077})

In [20]:
# prompt: 要求只在eval_dataset上evlauate模型

# ... (previous code remains the same)

# We can now start training.  We remove the training dataset here.
trainer = Trainer(
        model=model,
        #train_dataset=train_ds , # Removed training dataset
        eval_dataset=val_ds,
        data_collator=collate_fn,
        args=args
        )

# Evaluate the model
trainer.evaluate()

OutOfMemoryError: CUDA out of memory. Tried to allocate 7.95 GiB. GPU 0 has a total capacity of 39.56 GiB of which 7.18 GiB is free. Process 10867 has 32.28 GiB memory in use. Of the allocated memory 31.35 GiB is allocated by PyTorch, and 436.61 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [15]:
trainer.push_to_hub()

events.out.tfevents.1736933596.ea22571b339e.938.0:   0%|          | 0.00/15.9k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/45.3M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/testuayun/paligemma_vqav2/commit/8cac7d1d372b90a308f9481253fefdc5849a44af', commit_message='End of training', commit_description='', oid='8cac7d1d372b90a308f9481253fefdc5849a44af', pr_url=None, repo_url=RepoUrl('https://huggingface.co/testuayun/paligemma_vqav2', endpoint='https://huggingface.co', repo_type='model', repo_id='testuayun/paligemma_vqav2'), pr_revision=None, pr_num=None)

In [25]:
!nvidia-smi

Wed Jan 15 11:53:52 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              48W / 400W |  33121MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [26]:
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

40514

You can find steps to infer [here](https://colab.research.google.com/drive/100IQcvMvGm9y--oelbLfI__eHCoz5Ser?usp=sharing).